<a href="https://colab.research.google.com/github/soheldatta17/Intro-to-Data-Science/blob/main/text_data_pre_processing_use_case.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing the Dependencies

In [12]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [13]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:
# printing the stopwords
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Pre-Processing

In [24]:
news_data = pd.read_csv('/content/train.csv', encoding='latin1', on_bad_lines='skip', engine='python')


In [25]:
# first 5 rows of the dataset
news_data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didnât Even See Comeyâs...,Darrell Lucus,House Dem Aide: We Didnât Even See Comeyâs...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


0 --> Real News

1 --> Fake News

In [26]:
news_data.shape

(408, 5)

In [23]:
# checking for missing values
news_data.isnull().sum()

id         0
title      7
author    47
text       1
label      0
dtype: int64

In [27]:
# replacing the missing values with null string
news_data = news_data.fillna('')

In [28]:
# merging th eauthor name and news title
news_data['content'] = news_data['author']+' '+news_data['title']

In [29]:
# first 5 rows of the dataset
news_data.head()

,id,title,author,text,label,content
0,0,House Dem Aide: We Didnât Even See Comeyâs...,Darrell Lucus,House Dem Aide: We Didnât Even See Comeyâs...,1,Darrell Lucus House Dem Aide: We Didnât Even...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...


In [30]:
# separating feature and target
X = news_data.drop(columns='label', axis =1)
Y = news_data['label']

In [31]:
print(X)

      id                                              title  \
0      0  House Dem Aide: We Didnât Even See Comeyâs...   
1      1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2      2                  Why the Truth Might Get You Fired   
3      3  15 Civilians Killed In Single US Airstrike Hav...   
4      4  Iranian woman jailed for fictional unpublished...   
..   ...                                                ...   
403  403  Warriors, Resilient at Home, Cruise Against th...   
404  404  Comey Letter on Clinton Email Is Subject of Ju...   
405  405  Trump and Clinton Choice Impacting Global Markets   
406  406  Chinese J-20 stealth fighter debuts in skies o...   
407  407  Spain, Malta, under U.S./UK pressure, refuse t...   

                                           author  \
0                                   Darrell Lucus   
1                                 Daniel J. Flynn   
2                              Consortiumnews.com   
3                              

In [32]:
print(Y)

0      1
1      0
2      1
3      1
4      1
      ..
403    0
404    0
405    1
406    1
407    1
Name: label, Length: 408, dtype: int64


Stemming:

Stemming is the process of reducing a word to its Root Word

In [33]:
port_stem = PorterStemmer()

In [34]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ',content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

In [35]:
news_data['content'] = news_data['content'].apply(stemming)

In [36]:
print(news_data['content'])

0      darrel lucu hous dem aid even see comey letter...
1      daniel j flynn flynn hillari clinton big woman...
2                 consortiumnew com truth might get fire
3      jessica purkiss civilian kill singl us airstri...
4      howard portnoy iranian woman jail fiction unpu...
                             ...                        
403    scott cacciola warrior resili home cruis caval...
404    adam goldman eric lichtblau matt apuzzo comey ...
405             trump clinton choic impact global market
406            chines j stealth fighter debut sky zhuhai
407    j e dyer spain malta u uk pressur refus allow ...
Name: content, Length: 408, dtype: object


In [37]:
X = news_data['content'].values
Y = news_data['label'].values

In [38]:
print(X)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire'
 'jessica purkiss civilian kill singl us airstrik identifi'
 'howard portnoy iranian woman jail fiction unpublish stori woman stone death adulteri'
 'daniel nussbaum jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart'
 'life life luxuri elton john favorit shark pictur stare long transcontinent flight'
 'alissa j rubin beno hamon win french socialist parti presidenti nomin new york time'
 'excerpt draft script donald trump q ampa black church pastor new york time'
 'megan twohey scott shane back channel plan ukrain russia courtesi trump associ new york time'
 'aaron klein obama organ action partner soro link indivis disrupt trump agenda'
 'chri tomlinson bbc comedi sketch real housew isi caus outrag'
 'amando flavio russian research discov secret nazi militari base

In [39]:
print(Y)

[1 0 1 1 1 0 1 0 0 0 0 0 1 1 1 0 0 1 1 0 1 0 0 1 0 1 0 1 0 0 0 1 0 0 0 0 1
 1 0 0 0 1 0 1 0 0 1 1 0 0 1 1 0 1 1 1 1 1 1 0 0 1 0 1 0 0 1 1 0 1 1 1 0 0
 0 0 1 1 0 0 0 0 1 1 0 0 1 0 1 0 0 1 0 1 0 0 1 0 0 0 1 0 0 1 0 1 0 1 0 0 0
 0 1 0 1 0 0 0 0 0 1 1 1 0 1 1 1 0 0 0 0 1 0 0 1 1 0 1 1 0 1 1 1 1 1 1 1 0
 1 0 1 1 0 0 0 0 1 0 0 1 0 1 1 1 1 0 0 0 0 1 1 0 1 1 0 1 0 1 1 1 0 0 0 0 1
 0 1 0 0 1 1 0 1 1 1 1 1 1 1 0 0 1 0 1 1 0 0 1 0 1 1 0 1 0 1 1 1 0 0 0 0 1
 0 1 0 0 1 0 1 1 0 1 0 1 0 1 1 0 1 1 0 0 1 0 0 0 0 0 1 0 1 0 0 1 1 0 0 1 0
 0 1 1 0 0 0 0 1 1 0 0 0 0 0 1 0 0 1 1 1 1 0 1 0 1 1 1 0 0 1 0 1 0 1 0 1 1
 0 0 0 1 0 0 0 0 1 0 0 1 0 1 0 0 1 0 0 0 0 1 1 1 0 0 1 0 1 0 0 0 0 0 0 0 0
 1 0 1 1 1 1 0 0 1 0 1 1 1 0 1 1 0 1 0 0 1 0 0 1 0 0 0 1 1 1 1 1 0 0 1 0 0
 0 1 1 1 0 0 1 0 1 1 0 0 1 1 1 1 0 0 1 0 0 1 1 1 0 0 0 1 1 1 0 0 0 0 0 1 1
 1]


In [40]:
Y.shape

(408,)

In [41]:
# converting the textual data to feature vectors
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [42]:
print(X)

  (0, 1998)	0.28963050607718127
  (0, 1690)	0.30443684583210323
  (0, 1159)	0.30443684583210323
  (0, 1113)	0.28963050607718127
  (0, 996)	0.2687621422676085
  (0, 908)	0.2375657157096449
  (0, 644)	0.28963050607718127
  (0, 500)	0.28963050607718127
  (0, 474)	0.30443684583210323
  (0, 394)	0.2608283585614405
  (0, 322)	0.32530520964167603
  (0, 43)	0.28963050607718127
  (1, 2125)	0.28852484898790376
  (1, 888)	0.2142804685059584
  (1, 729)	0.6984513197799057
  (1, 471)	0.29859852689636235
  (1, 374)	0.23098216958114873
  (1, 288)	0.34922565988995286
  (1, 247)	0.1713754985747369
  (1, 203)	0.29859852689636235
  (2, 1989)	0.42677410929768966
  (2, 1235)	0.4485914328471016
  (2, 782)	0.37420699527227974
  (2, 714)	0.38433379105091164
  (2, 414)	0.4485914328471016
  :	:
  (404, 20)	0.2729942143778337
  (405, 1986)	0.20546924487353838
  (405, 1186)	0.44972118410581485
  (405, 942)	0.44972118410581485
  (405, 794)	0.43188843518233977
  (405, 374)	0.3340891491070496
  (405, 351)	0.505114761